# Regresja

Dane wykorzystywane w pierwszej części ćwiczeń pochodzą z materiałów do książki: [F. Nwanganga, M. Chapple: Praktyczne uczenie maszynowe w języku R]("https://www.wiley.com/en-ie/Practical+Machine+Learning+in+R-p-9781119591511").

## 1. Prosta regresja liniowa

### Wczytanie danych

Pobierz dane `bikes.csv` do swojego katalogu roboczego, a następnie wczytaj je. Użyj argumentu `col_types` żeby ustawić typ poszczególnych kolumn: `D` - data, `f` - faktor, `d` - double.

In [ ]:
library(tidyverse)
rowery <- read_csv("./bikes.csv", col_types = "Dffffddddd")

Wyświetl podgląd danych za pomocą funkcji `glimpse()`

In [ ]:
glimpse(rowery)

Wyświetl i przeanalizuj statystyki opisowe danych `rowery`.

In [ ]:
summary(rowery)

Narysuj wykresy punktowe porównujące `rentals` (liczbę wypożyczonych rowerów) do trzech innych zmiennych (`humidity`, `windspeed` oraz `temperature`). Co możesz powiedzieć na temat zależności między zmienną `rentals` a pozostałymi wybranymi zmiennymi na podstawie samych wykresów?

In [ ]:
#library(ggplot2)

#df <- data.frame(x,y)
humid <- ggplot(data=rowery, aes(x=humidity, y=rentals)) + geom_point()
wind <-  ggplot(data=rowery, aes(x=windspeed, y=rentals)) + geom_point()
temp <-  ggplot(data=rowery, aes(x=temperature, y=rentals)) + geom_point()

library(gridExtra)
grid.arrange(humid, wind, temp, nrow=1, ncol=3)

### Korelacja

Oblicz współczynnik korelacji liniowej Pearsona pomiędzy zmiennymi `rentals` i `humidity`.

In [ ]:
cov(rowery$rentals, rowery$humidity)

sd(rowery$rentals)
sd(rowery$humidity)

pearson <- cov(rowery$rentals, rowery$humidity) / 
  (sd(rowery$rentals) * sd(rowery$humidity))

pearson

Jaki wniosek można wyciągnąć na temat zależności pomiędzy badanymi zmiennymi na podstawie obliczonego współczynnika korelacji Pearsona?

Oblicz współczynnik korelacji Pearsona między zmienną `rentals` a zmiennymi `temperature` oraz `windspeed`, wykorzystując funkcję `cor()`.

In [ ]:
cor(rowery$temperature, rowery$rentals)
cor(rowery$windspeed, rowery$rentals)

Na podstawie obliczonych współczynników korelacji odpowiedz na pytania:

- Czy istnieje jakaś zależność między liczbą wypożyczonych rowerów a pozostałymi zmiennymi?

- Jeśli istnieje zależność, to jak silna?

Oblicz tabelę współczynników korelacji dla  dla zmiennych liczbowych ze zbioru `rowery`. Następnie zwizualizuj korelacje za pomocą funkcji `corrplot`.

In [ ]:
rowery_liczbowe <- rowery %>%
  select(-(date:weather))

rowery_korelacje <- cor(rowery_liczbowe)

library(corrplot)
corrplot(rowery_korelacje)

Ponieważ korelacja między A i B jest taka sama, jak pomiędzy B i A, możemy uprościć wizualizację, pokazując tylko współczynniki nad przekątną - służy do tego argument `type` funkcji `corrplot()`. Ponadto za pomocą argumentu `method` możemy wyświetlić wartości współczynników zamiast kółek.

In [ ]:
corrplot(rowery_korelacje, type = "upper", method = "number")

### Współczynniki prostej regresji

Ręcznie uzyskaj wartość współczynników `b0` i `b1`.

In [ ]:
b1 <- cov(rowery$temperature, rowery$rentals) / var(rowery$temperature)
b1

b0 <- mean(rowery$rentals) - b1 * mean(rowery$temperature)
b0

### Model prostej regresji liniowej

Zbuduj model liniowy dla pary zmiennych `rentals` oraz `temperature`.

In [ ]:
rowery_model1 <- lm(data = rowery, rentals~temperature)
rowery_model1

Wyświetl bardziej szczegółowe informacje o modelu za pomocą funkcji `summary()`.

In [ ]:
summary(rowery_model1)

Co oznaczają statystyki podsumowujące dla reszt (sekcja `Residuals`)?

## 2. Wielokrotna regresja liniowa

Model uzyskany za pomocą pojedynczego predyktora (`temperature`) tłumaczył około 39% zmienności w zbiorze danych.  W kolejnym modelu uwzględnij dodatkowo zmienne `windspeed` oraz `humidity`.

In [ ]:
rowery_model2 <- lm(data = rowery, rentals~humidity+windspeed+temperature)

summary(rowery_model2)

Jak oceniasz nowo powstały model? 

Napisz równanie w jaki sposób `rentals` zależy od wybranych predyktorów.


## 3. Zadania

1. Pobierz i wczytaj zbiór danych `Advertising.csv`.

2. Odpowiedz na pytania: 

- czy istnieje zależność pomiędzy sprzedażą a budżetem reklamowym dla usług `TV`, `radio` oraz `newspaper`.
- jeśli istnieje - jak silna jest ta zależność?
- które media są powiązane ze sprzedażą?
- jak duża jest zależność pomiędzy poszczególnymi mediami a sprzedażą?
- czy zależność jest liniowa?